In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from core.RobotLink import *
from core.StereoCamera import *
from core.ParticleFilter import *
from core.probability_functions import *
from core.utils import *
import matplotlib.pyplot as plt

import time
import rosbag

Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


In [2]:
psm_arm = RobotLink('../journal_dataset/LND.json')
cam = StereoCamera('../journal_dataset/camera_calibration.yaml', rectify=True)

In [3]:
# Get a sample data!!
bag = rosbag.Bag('../journal_dataset/stationary_camera_2020-06-24-15-49-10.bag')

ecm_pose_list  = []
ecm_pose_ts    = []
ecm_joint_list = []
ecm_joint_ts   = []
psm_joint_list = []
psm_joint_ts   = []
psm_jaw_list   = []
psm_jaw_ts     = []
left_img_list  = []
left_img_ts    = []
right_img_list = []
right_img_ts   = []

for topic, msg, t in bag.read_messages(topics=['/dvrk/ECM/position_cartesian_current', 
                                               '/dvrk/ECM/state_joint_current',
                                               '/dvrk/PSM1/state_joint_current',
                                               '/dvrk/PSM1/state_jaw_current',
                                               '/stereo/left/image',
                                               '/stereo/right/image']):
    if topic == '/dvrk/ECM/position_cartesian_current':
        ecm_pose_list.append(msg)
        ecm_pose_ts.append(msg.header.stamp.to_sec())
    elif topic == '/dvrk/ECM/state_joint_current':
        ecm_joint_list.append(msg)
        ecm_joint_ts.append(msg.header.stamp.to_sec())
    elif topic == '/dvrk/PSM1/state_joint_current':
        psm_joint_list.append(msg)
        psm_joint_ts.append(msg.header.stamp.to_sec())
    elif topic == '/dvrk/PSM1/state_jaw_current':
        psm_jaw_list.append(msg)
        psm_jaw_ts.append(msg.header.stamp.to_sec())
    elif topic == '/stereo/left/image':
        left_img_list.append(msg)
        left_img_ts.append(msg.header.stamp.to_sec())
    elif topic == '/stereo/right/image':
        right_img_list.append(msg)
        right_img_ts.append(msg.header.stamp.to_sec())
    
    # if len(ecm_pose_list) == 1000:
    #     break

bag.close()

ecm_pose_ts  = np.array(ecm_pose_ts)
ecm_joint_ts = np.array(ecm_joint_ts)
psm_joint_ts = np.array(psm_joint_ts)
psm_jaw_ts   = np.array(psm_jaw_ts)
left_img_ts  = np.array(left_img_ts)
right_img_ts = np.array(right_img_ts)

In [4]:
# Load hand-eye transform (this will be hard-coded for now according to the dataset)
f = open('../journal_dataset/handeye.yaml')
hand_eye_data = yaml.load(f, Loader=yaml.FullLoader)

bc_T_cam = np.eye(4)
bc_T_cam[:-1, -1] = np.array(hand_eye_data['cam_tvec'])/1000.0
bc_T_cam[:-1, :-1] = axisAngleToRotationMatrix(hand_eye_data['cam_rvec'])

cam_T_b = np.eye(4)
cam_T_b[:-1, -1] = np.array(hand_eye_data['PSM1_tvec'])/1000.0
cam_T_b[:-1, :-1] = axisAngleToRotationMatrix(hand_eye_data['PSM1_rvec'])

In [9]:
# initialize filter
pf = ParticleFilter(num_states=6,
                    initialDistributionFunc=sampleNormalDistribution,
                    motionModelFunc=additiveGaussianNoise,
                    obsModelFunc=pointFeatureObs,
                    num_particles=200)

initalize = True


# output video
left_video  = cv2.VideoWriter("left_video.mp4",  cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), 30, cam.img_size)
right_video = cv2.VideoWriter("right_video.mp4", cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), 30, cam.img_size)


for left_msg_idx, left_img_msg in enumerate(left_img_list):
    # Synchronize the data to the left_img_msg
    ts = left_img_ts[left_msg_idx]

    ecm_pose_msg  = ecm_pose_list [np.argmin(np.abs(ecm_pose_ts - ts))]
    ecm_joint_msg = ecm_joint_list[np.argmin(np.abs(ecm_joint_ts - ts))]
    psm_joint_msg = psm_joint_list[np.argmin(np.abs(psm_joint_ts - ts))]
    psm_jaw_msg   = psm_jaw_list  [np.argmin(np.abs(psm_jaw_ts - ts))]
    # left_img_msg  = left_img_list[ left_msg_idx]
    right_img_msg = right_img_list[np.argmin(np.abs(right_img_ts - ts))]
    
    try:
        left_img  = np.ndarray(shape=(left_img_msg.height, left_img_msg.width, 3),
                                      dtype=np.uint8, buffer=left_img_msg.data)
        right_img = np.ndarray(shape=(right_img_msg.height, right_img_msg.width, 3),
                                      dtype=np.uint8, buffer=right_img_msg.data)
    except:
        continue

    left_img, right_img = cam.processImage(left_img, right_img)
    
    # Detect segmented keypoints and get new joint angle readings
    detected_keypoints_l, left_img  = segmentColorAndGetKeyPoints(left_img,  draw_contours=True)
    detected_keypoints_r, right_img = segmentColorAndGetKeyPoints(right_img, draw_contours=True)
    new_joint_angles = np.array(psm_joint_msg.position + psm_jaw_msg.position)

    start_t = time.time()
    psm_arm.updateJointAngles(new_joint_angles)
    
    if initalize:
        # Initialize particle filter
        initalize = False
        
        init_kwargs = {
                        "std": np.array([1.0e-3, 1.0e-3, 1.0e-3, # pos
                                         1.0e-2, 1.0e-2, 1.0e-2, # ori
                                        ])
                      }
        pf.initializeFilter(**init_kwargs)

    else:
        # Predict particle filter
        pred_kwargs = {
                        "std": np.array([2.5e-5, 2.5e-5, 2.5e-5, 
                                         1.0e-4, 1.0e-4, 1.0e-4])
                      }
        pf.predictionStep(**pred_kwargs)
    
    # Update particle filter
    upd_kwargs = {
                    "point_detections": (detected_keypoints_l, detected_keypoints_r), 
                    "robot_arm": psm_arm, 
                    "cam": cam, 
                    "cam_T_b": cam_T_b,
                    "joint_angle_readings": new_joint_angles,
                    "gamma": 0.15
    }
    
    pf.updateStep(**upd_kwargs)
    last_joint_angle_reading = new_joint_angles
    
    correction_estimation = pf.getMeanParticle()
    
    print("Time to update robot: {}ms".format((time.time()-start_t)*1000))
    
    # Project skeleton
    T = poseToMatrix(correction_estimation)  
    psm_arm.updateJointAngles(new_joint_angles)
    
    img_list = projectSkeleton(psm_arm.getSkeletonPoints(), np.dot(cam_T_b, T),
                               [left_img, right_img], cam.projectPoints)
    
    left_video.write( img_list[0])
    right_video.write(img_list[1])
    
left_video.release()
right_video.release()

Time to update robot: 48.65527153015137ms
Time to update robot: 46.30613327026367ms
Time to update robot: 44.79384422302246ms
Time to update robot: 44.48390007019043ms
Time to update robot: 46.950578689575195ms
Time to update robot: 45.458078384399414ms
Time to update robot: 44.07835006713867ms
Time to update robot: 44.20733451843262ms
Time to update robot: 48.5384464263916ms
Time to update robot: 45.84836959838867ms
Time to update robot: 44.23260688781738ms
Time to update robot: 45.064687728881836ms
Time to update robot: 49.8659610748291ms
Time to update robot: 46.222686767578125ms
Time to update robot: 50.752878189086914ms
Time to update robot: 54.49867248535156ms
Time to update robot: 50.6591796875ms
Time to update robot: 48.42519760131836ms
Time to update robot: 48.65288734436035ms
Time to update robot: 44.68989372253418ms
Time to update robot: 45.238494873046875ms
Time to update robot: 44.198036193847656ms
Time to update robot: 44.6317195892334ms
Time to update robot: 51.848888397

Time to update robot: 47.011613845825195ms
Time to update robot: 44.65198516845703ms
Time to update robot: 44.49915885925293ms
Time to update robot: 45.94731330871582ms
Time to update robot: 48.48527908325195ms
Time to update robot: 46.88096046447754ms
Time to update robot: 45.732975006103516ms
Time to update robot: 47.95718193054199ms
Time to update robot: 47.30820655822754ms
Time to update robot: 44.96502876281738ms
Time to update robot: 45.621395111083984ms
Time to update robot: 48.77662658691406ms
Time to update robot: 46.54359817504883ms
Time to update robot: 45.14336585998535ms
Time to update robot: 47.890424728393555ms
Time to update robot: 48.53963851928711ms
Time to update robot: 48.83003234863281ms
Time to update robot: 47.730207443237305ms
Time to update robot: 45.77803611755371ms
Time to update robot: 48.10190200805664ms
Time to update robot: 50.12249946594238ms
Time to update robot: 47.85966873168945ms
Time to update robot: 47.60265350341797ms
Time to update robot: 46.2975

Time to update robot: 47.8816032409668ms
Time to update robot: 45.020103454589844ms
Time to update robot: 43.76578330993652ms
Time to update robot: 44.500112533569336ms
Time to update robot: 47.03354835510254ms
Time to update robot: 44.4643497467041ms
Time to update robot: 43.46776008605957ms
Time to update robot: 44.45981979370117ms
Time to update robot: 49.01862144470215ms
Time to update robot: 45.635223388671875ms
Time to update robot: 43.58243942260742ms
Time to update robot: 45.479536056518555ms
Time to update robot: 48.53177070617676ms
Time to update robot: 45.7913875579834ms
Time to update robot: 47.30820655822754ms
Time to update robot: 45.623064041137695ms
Time to update robot: 47.74212837219238ms
Time to update robot: 45.963287353515625ms
Time to update robot: 44.573307037353516ms
Time to update robot: 46.00048065185547ms
Time to update robot: 47.60479927062988ms
Time to update robot: 45.0742244720459ms
Time to update robot: 44.999122619628906ms
Time to update robot: 44.56448

Time to update robot: 48.127174377441406ms
Time to update robot: 45.905113220214844ms
Time to update robot: 44.4486141204834ms
Time to update robot: 45.11737823486328ms
Time to update robot: 47.220706939697266ms
Time to update robot: 44.07358169555664ms
Time to update robot: 45.52316665649414ms
Time to update robot: 44.04020309448242ms
Time to update robot: 48.40254783630371ms
Time to update robot: 46.26607894897461ms
Time to update robot: 45.85456848144531ms
Time to update robot: 47.75190353393555ms
Time to update robot: 46.35119438171387ms
Time to update robot: 43.91932487487793ms
Time to update robot: 46.54741287231445ms
Time to update robot: 45.20726203918457ms
Time to update robot: 46.61273956298828ms
Time to update robot: 45.59588432312012ms
Time to update robot: 44.737815856933594ms
Time to update robot: 46.602725982666016ms
Time to update robot: 47.385215759277344ms
Time to update robot: 51.8341064453125ms
Time to update robot: 45.77898979187012ms
Time to update robot: 46.61989

Time to update robot: 44.62432861328125ms
Time to update robot: 42.504072189331055ms
Time to update robot: 41.2294864654541ms
Time to update robot: 43.54381561279297ms
Time to update robot: 44.62623596191406ms
Time to update robot: 40.95888137817383ms
Time to update robot: 42.46068000793457ms
Time to update robot: 41.06426239013672ms
Time to update robot: 48.09236526489258ms
Time to update robot: 41.35274887084961ms
Time to update robot: 40.88091850280762ms
Time to update robot: 42.017459869384766ms
Time to update robot: 47.06168174743652ms
Time to update robot: 45.91035842895508ms
Time to update robot: 46.65994644165039ms
Time to update robot: 43.70737075805664ms
Time to update robot: 50.301313400268555ms
Time to update robot: 44.356584548950195ms
Time to update robot: 42.02389717102051ms
Time to update robot: 43.42460632324219ms
Time to update robot: 45.23801803588867ms
Time to update robot: 41.60809516906738ms
Time to update robot: 44.7087287902832ms
Time to update robot: 45.2167987

Time to update robot: 47.098398208618164ms
Time to update robot: 44.54493522644043ms
Time to update robot: 44.959068298339844ms
Time to update robot: 43.35832595825195ms
Time to update robot: 50.88353157043457ms
Time to update robot: 43.259382247924805ms
Time to update robot: 44.43359375ms
Time to update robot: 43.14684867858887ms
Time to update robot: 51.316022872924805ms
Time to update robot: 46.74696922302246ms
Time to update robot: 44.64268684387207ms
Time to update robot: 44.252872467041016ms
Time to update robot: 48.04086685180664ms
Time to update robot: 44.11053657531738ms
Time to update robot: 44.2962646484375ms
Time to update robot: 44.7382926940918ms
Time to update robot: 45.41802406311035ms
Time to update robot: 44.46911811828613ms
Time to update robot: 43.30301284790039ms
Time to update robot: 44.72804069519043ms
Time to update robot: 47.47962951660156ms
Time to update robot: 42.255401611328125ms
Time to update robot: 46.282291412353516ms
Time to update robot: 43.8089370727

Time to update robot: 46.53310775756836ms
Time to update robot: 44.582366943359375ms
Time to update robot: 46.89526557922363ms
Time to update robot: 44.01040077209473ms
Time to update robot: 47.32370376586914ms
Time to update robot: 46.01621627807617ms
Time to update robot: 44.091224670410156ms
Time to update robot: 44.1281795501709ms
Time to update robot: 47.94740676879883ms
Time to update robot: 44.55876350402832ms
Time to update robot: 45.310258865356445ms
Time to update robot: 43.257951736450195ms
Time to update robot: 49.349069595336914ms
Time to update robot: 47.608137130737305ms
Time to update robot: 45.06039619445801ms
Time to update robot: 48.3698844909668ms
Time to update robot: 45.877933502197266ms
Time to update robot: 45.12596130371094ms
Time to update robot: 49.85928535461426ms
Time to update robot: 43.50757598876953ms
Time to update robot: 46.80132865905762ms
Time to update robot: 48.6452579498291ms
Time to update robot: 43.54524612426758ms
Time to update robot: 42.67740

Time to update robot: 49.292564392089844ms
Time to update robot: 45.01700401306152ms
Time to update robot: 46.76508903503418ms
Time to update robot: 44.948577880859375ms
Time to update robot: 48.66504669189453ms
Time to update robot: 46.20814323425293ms
Time to update robot: 45.16863822937012ms
Time to update robot: 46.24152183532715ms
Time to update robot: 47.41811752319336ms
Time to update robot: 44.76475715637207ms
Time to update robot: 45.99189758300781ms
Time to update robot: 46.01287841796875ms
Time to update robot: 47.585487365722656ms
Time to update robot: 46.770572662353516ms
Time to update robot: 45.86648941040039ms
Time to update robot: 46.28348350524902ms
Time to update robot: 48.137664794921875ms
Time to update robot: 45.256853103637695ms
Time to update robot: 46.20862007141113ms
Time to update robot: 45.21751403808594ms
Time to update robot: 46.13995552062988ms
Time to update robot: 47.41811752319336ms
Time to update robot: 46.027421951293945ms
Time to update robot: 46.93

Time to update robot: 49.263954162597656ms
Time to update robot: 45.012474060058594ms
Time to update robot: 45.29070854187012ms
Time to update robot: 43.55978965759277ms
Time to update robot: 47.93286323547363ms
Time to update robot: 45.81713676452637ms
Time to update robot: 44.519662857055664ms
Time to update robot: 44.93308067321777ms
Time to update robot: 49.27492141723633ms
Time to update robot: 45.41015625ms
Time to update robot: 46.38171195983887ms
Time to update robot: 45.027732849121094ms
Time to update robot: 45.70484161376953ms
Time to update robot: 44.86584663391113ms
Time to update robot: 43.79725456237793ms
Time to update robot: 44.014930725097656ms
Time to update robot: 48.728227615356445ms
Time to update robot: 44.84200477600098ms
Time to update robot: 45.81594467163086ms
Time to update robot: 44.695377349853516ms
Time to update robot: 49.448251724243164ms
Time to update robot: 55.554866790771484ms
Time to update robot: 44.0821647644043ms
Time to update robot: 44.9604988

Time to update robot: 47.81341552734375ms
Time to update robot: 49.60989952087402ms
Time to update robot: 45.9744930267334ms
Time to update robot: 44.28267478942871ms
Time to update robot: 46.22483253479004ms
Time to update robot: 44.22497749328613ms
Time to update robot: 43.90978813171387ms
Time to update robot: 44.336795806884766ms
Time to update robot: 51.44667625427246ms
Time to update robot: 46.68879508972168ms
Time to update robot: 43.06316375732422ms
